In [54]:
# data science
import pandas as pd
import numpy as np

# machine learning
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import CategoricalNB, GaussianNB
from sklearn import preprocessing
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC

from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

# viz
#import graphviz


# 0. Read csv and clean data

In [55]:
df = pd.read_csv("./EEG_data.csv")
df

,SubjectID,VideoID,Attention,Mediation,Raw,Delta,Theta,Alpha1,Alpha2,Beta1,Beta2,Gamma1,Gamma2,predefinedlabel,user-definedlabeln
0,0.0,0.0,56.0,43.0,278.0,301963.0,90612.0,33735.0,23991.0,27946.0,45097.0,33228.0,8293.0,0.0,0.0
1,0.0,0.0,40.0,35.0,-50.0,73787.0,28083.0,1439.0,2240.0,2746.0,3687.0,5293.0,2740.0,0.0,0.0
2,0.0,0.0,47.0,48.0,101.0,758353.0,383745.0,201999.0,62107.0,36293.0,130536.0,57243.0,25354.0,0.0,0.0
3,0.0,0.0,47.0,57.0,-5.0,2012240.0,129350.0,61236.0,17084.0,11488.0,62462.0,49960.0,33932.0,0.0,0.0
4,0.0,0.0,44.0,53.0,-8.0,1005145.0,354328.0,37102.0,88881.0,45307.0,99603.0,44790.0,29749.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12806,9.0,9.0,64.0,38.0,-39.0,127574.0,9951.0,709.0,21732.0,3872.0,39728.0,2598.0,960.0,1.0,0.0
12807,9.0,9.0,61.0,35.0,-275.0,323061.0,797464.0,153171.0,145805.0,39829.0,571280.0,36574.0,10010.0,1.0,0.0
12808,9.0,9.0,60.0,29.0,-426.0,680989.0,154296.0,40068.0,39122.0,10966.0,26975.0,20427.0,2024.0,1.0,0.0
12809,9.0,9.0,60.0,29.0,-84.0,366269.0,27346.0,11444.0,9932.0,1939.0,3283.0,12323.0,1764.0,1.0,0.0


### Cleaning

In [56]:
df = df.drop(["SubjectID", "VideoID", "user-definedlabeln"], axis=1)
df = df[df["predefinedlabel"].isin([0, 1])]
for col in df.columns:
    if col == 'predefinedlabel': continue
    #df[col] = (df[col] - df[col].mean()) / df[col].std() 
    df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())
df

,Attention,Mediation,Raw,Delta,Theta,Alpha1,Alpha2,Beta1,Beta2,Gamma1,Gamma2,predefinedlabel
0,0.56,0.43,0.568010,0.076059,0.030120,0.024623,0.023590,0.026169,0.027407,0.016845,0.006150,0.0
1,0.40,0.35,0.487912,0.018500,0.009331,0.001049,0.002201,0.002569,0.002240,0.002683,0.002031,0.0
2,0.47,0.48,0.524786,0.191187,0.127578,0.147448,0.061072,0.033987,0.079334,0.029020,0.018806,0.0
3,0.47,0.57,0.498901,0.507488,0.042999,0.044698,0.016798,0.010756,0.037961,0.025328,0.025168,0.0
4,0.44,0.53,0.498168,0.253442,0.117798,0.027081,0.087401,0.042428,0.060534,0.022707,0.022066,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
12806,0.64,0.38,0.490598,0.032068,0.003303,0.000516,0.021369,0.003623,0.024144,0.001317,0.000711,1.0
12807,0.61,0.35,0.432967,0.081381,0.265128,0.111806,0.143378,0.037298,0.347204,0.018541,0.007424,1.0
12808,0.60,0.29,0.396093,0.171671,0.051293,0.029246,0.038469,0.010267,0.016393,0.010355,0.001500,1.0
12809,0.60,0.29,0.479609,0.092281,0.009086,0.008352,0.009765,0.001813,0.001994,0.006247,0.001307,1.0


### Preprocessing

In [57]:
# Random shuffle so that labels are not clustered
df = df.sample(frac=1, random_state = 1).reset_index()
df

,index,Attention,Mediation,Raw,Delta,Theta,Alpha1,Alpha2,Beta1,Beta2,Gamma1,Gamma2,predefinedlabel
0,8324,0.00,0.00,0.417827,0.293120,0.153390,0.015698,0.039047,0.046898,0.101441,0.091757,0.029561,0.0
1,11677,0.48,0.51,0.489377,0.254506,0.044620,0.013299,0.036181,0.011870,0.006721,0.007809,0.001678,0.0
2,343,0.70,0.40,0.505495,0.231240,0.022648,0.008634,0.004287,0.022071,0.041064,0.019772,0.016140,0.0
3,9726,0.40,0.77,0.456654,0.434374,0.185498,0.066741,0.067700,0.031332,0.015539,0.010508,0.036084,1.0
4,1505,0.51,0.87,0.534554,0.187854,0.011714,0.015411,0.007623,0.009701,0.005064,0.002051,0.000499,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12806,10955,0.61,0.69,0.517216,0.431144,0.063426,0.119992,0.150074,0.067566,0.044352,0.021630,0.014327,1.0
12807,905,0.43,0.38,0.468376,0.062383,0.040838,0.024344,0.012618,0.013018,0.016410,0.007233,0.004718,1.0
12808,5192,0.64,0.54,0.498168,0.104693,0.006972,0.004961,0.006049,0.002269,0.002659,0.000577,0.000385,0.0
12809,12172,0.24,0.53,0.481563,0.198073,0.034533,0.013238,0.044091,0.008556,0.031107,0.022799,0.000841,0.0


In [58]:
# Train : Test = 8:2
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)
n_objects = df.shape[0]
n_train = int(n_objects * 0.8)
df_train = df.iloc[:n_train]
df_test = df.iloc[n_train:]
df_train

,index,Attention,Mediation,Raw,Delta,Theta,Alpha1,Alpha2,Beta1,Beta2,Gamma1,Gamma2,predefinedlabel
0,8324,0.00,0.00,0.417827,0.293120,0.153390,0.015698,0.039047,0.046898,0.101441,0.091757,0.029561,0.0
1,11677,0.48,0.51,0.489377,0.254506,0.044620,0.013299,0.036181,0.011870,0.006721,0.007809,0.001678,0.0
2,343,0.70,0.40,0.505495,0.231240,0.022648,0.008634,0.004287,0.022071,0.041064,0.019772,0.016140,0.0
3,9726,0.40,0.77,0.456654,0.434374,0.185498,0.066741,0.067700,0.031332,0.015539,0.010508,0.036084,1.0
4,1505,0.51,0.87,0.534554,0.187854,0.011714,0.015411,0.007623,0.009701,0.005064,0.002051,0.000499,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10243,8841,0.00,0.00,0.000000,0.075209,0.104573,0.016795,0.032925,0.016195,0.013742,0.011037,0.022783,1.0
10244,2323,0.47,0.69,0.502808,0.036662,0.011418,0.012376,0.007118,0.006136,0.001964,0.000978,0.000975,1.0
10245,9162,0.78,0.56,0.498168,0.103168,0.008186,0.003961,0.006533,0.002070,0.007223,0.002955,0.006338,0.0
10246,7186,0.47,0.44,0.509890,0.036579,0.057042,0.001124,0.004309,0.004168,0.003815,0.001982,0.000539,1.0


In [59]:
def encode(df_to_encode):
    le = preprocessing.LabelEncoder()

    df_encoded = df_to_encode.copy()
#     df_encoded["Attention"] = le.fit_transform(df_encoded["Attention"])
#     df_encoded["Mediation"] = le.fit_transform(df_encoded["Mediation"])
#     df_encoded["Delta"] = le.fit_transform(df_encoded["Delta"])
#     df_encoded["Theta"] = le.fit_transform(df_encoded["Theta"])
#     df_encoded["Alpha1"] = le.fit_transform(df_encoded["Alpha1"])
#     df_encoded["Alpha2"] = le.fit_transform(df_encoded["Alpha2"])
#     df_encoded["Beta1"] = le.fit_transform(df_encoded["Beta1"])
#     df_encoded["Beta2"] = le.fit_transform(df_encoded["Beta2"])
#     df_encoded["Gamma1"] = le.fit_transform(df_encoded["Gamma1"])
#     df_encoded["Gamma2"] = le.fit_transform(df_encoded["Gamma2"])
#     df_encoded["Raw"] = le.fit_transform(df_encoded["Raw"])
#     df_encoded["predefinedlabel"] = le.fit_transform(df_encoded["predefinedlabel"])

    x_encoded = df_encoded[["Attention", "Mediation", "Raw", "Delta", "Theta", "Alpha1", "Alpha2", "Beta1", "Beta2", "Gamma1", "Gamma2"]]
    y_encoded = df_encoded["predefinedlabel"]
    
    return (x_encoded, y_encoded)

# 1. KNN

In [60]:
x_train, y_train = encode(df_train)
x_test, y_test = encode(df_test)

clf = KNeighborsClassifier()        # n_neighbors=5
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print("Predicted:", y_pred)
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

Predicted: [1. 1. 1. ... 1. 1. 1.]
Accuracy: 0.5540382364416699


# 2. Naive Bayes

In [61]:
clf = GaussianNB()#CategoricalNB()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print("Predicted:", y_pred)
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

y_pred_bayes = y_pred

Predicted: [1. 1. 1. ... 1. 1. 1.]
Accuracy: 0.49590323839250877


# 3. Decision Tree

In [62]:
# ID3
# "entropy" for the information gain
clf = tree.DecisionTreeClassifier(criterion="entropy", max_depth=9)
clf.fit(x_train, y_train)
dot_data = tree.export_graphviz(clf, out_file=None,
                                feature_names=["Attention", "Mediation", "Raw", "Delta", "Theta", "Alpha1", "Alpha2", "Beta1", "Beta2", "Gamma1", "Gamma2"],
                                class_names="Confusion",
                                filled=True, rounded=True,
                                special_characters=True)  
# graph = graphviz.Source(dot_data)
# graph
y_pred = clf.predict(x_test)
print("Predicted:", y_pred)
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

Predicted: [0. 0. 0. ... 1. 0. 0.]
Accuracy: 0.5450643776824035


# 4. SVM

In [63]:
# rbf kernel
clf = SVC()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print("Predicted:", y_pred)
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

Predicted: [0. 1. 1. ... 0. 0. 1.]
Accuracy: 0.5485758876316816


In [64]:
#linear
clf = LinearSVC()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print("Predicted:", y_pred)
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

Predicted: [0. 0. 0. ... 0. 0. 0.]
Accuracy: 0.5364806866952789


# Some Other Classification Models

In [65]:
clf_cart = tree.DecisionTreeClassifier(max_depth=2)
clf_forest = RandomForestClassifier()
clf_boost = AdaBoostClassifier(base_estimator=tree.DecisionTreeClassifier(max_depth=2), random_state=21)
clf_mlp = MLPClassifier(hidden_layer_sizes=(50,50,), max_iter=1000, tol=0.001, random_state=42)

clf_cart.fit(x_train, y_train)
clf_forest.fit(x_train, y_train)
clf_boost.fit(x_train, y_train)
clf_mlp.fit(x_train, y_train)

print("Accuracy:")
print("- CART:", metrics.accuracy_score(y_test, clf_cart.predict(x_test)))
print("- Random Forest:", metrics.accuracy_score(y_test, clf_forest.predict(x_test)))
print("- AdaBoost:", metrics.accuracy_score(y_test, clf_boost.predict(x_test)))
print("- MLP:", metrics.accuracy_score(y_test, clf_mlp.predict(x_test)))

Accuracy:
- CART: 0.5294576667967226
- Random Forest: 0.6199765899336714
- AdaBoost: 0.5661334373780725
- MLP: 0.5536480686695279
